In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os

if "DATABRICKS_RUNTIME_VERSION" in os.environ and not 'installed_libs' in globals():
  #CUDA = 'cu121' 
  installed_libs = True
  
  
  !pip install torch==2.1.0  torchvision==0.16.0 torchtext==0.16.0 torchaudio==2.1.0 --index-url https://download.pytorch.org/whl/cu121
  import torch
  #os.environ['TORCH'] = torch.__version__
  #print(torch.__version__)
  #torch_version = '2.0.0+cu118'
  
  #!pip install pyg_lib torch_scatter torch_sparse torch_cluster -f https://data.pyg.org/whl/torch-2.1.0+${CUDA}.html # torch_spline_conv
  !pip install torch_geometric
  !pip install pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.1.0+cu121.html
  #!pip install torch_sparse -f https://data.pyg.org/whl/torch-2.1.0+${CUDA}.html
  #!pip install torch_scatter -f https://data.pyg.org/whl/torch-2.1.0+${CUDA}.html
  #!pip install pyg_lib -f https://data.pyg.org/whl/torch-2.1.0+${CUDA}.html
  !pip install sentence-transformers
  !pip install torcheval
  !pip install matplotlib
  !pip install pandas
  !pip install tensorboard
  
if "DATABRICKS_RUNTIME_VERSION" in os.environ:
  ROOT_FOLDER = '/dbfs/FileStore/GraphNeuralNetworks/'
else:
  ROOT_FOLDER = ''

In [3]:
# sampler

def nf_sampler(batch_size, neg_sample_ratio, edge_label_index, num_learnings):
    # triplet mode only
    # sample some random edges
    num_samples = batch_size
    #sampled_indices = torch.randint(0, edge_label_index.shape[1], (num_samples,), replacement=False)
    sampled_indices = torch.randperm(edge_label_index.shape[1])[:num_samples]
    sampled_edges = edge_label_index[:, sampled_indices]
    # remove sampled edges from edge_label_index with mask
    mask = torch.ones(edge_label_index.shape[1], dtype=torch.bool)
    mask[sampled_indices] = False
    edge_label_index = edge_label_index[:, mask]
    
    assert neg_sample_ratio >= 1
    neg_samples = sampled_indices.shape[0]*neg_sample_ratio
    
    s = sampled_edges[0,:].unsqueeze(0)
    src_edges = s
    for i in range(neg_sample_ratio-1):
        src_edges= torch.cat((src_edges,s),dim=1)

    sampled_negatives = torch.randint(0, num_learnings, (neg_samples,)).squeeze().unsqueeze(0)
   
   
    negative_edge_label_index = torch.cat((src_edges, sampled_negatives),dim=0)
    edge_label_indices = torch.cat((sampled_edges, negative_edge_label_index),dim=1)
    new_edge_label_index = edge_label_index
    batch_edge_label_index = edge_label_indices
    return new_edge_label_index, batch_edge_label_index, torch.cat((torch.ones(batch_size), torch.zeros(neg_samples)))

def nf_loader(edge_label_index, batch_size, num_learnings, neg_sample_ratio):
    while edge_label_index.shape[1] > 0:
        new_edge_label_index, batch_edge_label_index, batch_labels = nf_sampler(batch_size, neg_sample_ratio, edge_label_index, num_learnings)
        edge_label_index = new_edge_label_index
        yield batch_edge_label_index, batch_labels
    

def get_total_minibatch_count_fm(batch_size, edge_label_index):
    return int((edge_label_index.shape[1]+batch_size)//batch_size)
import torch
from torch_geometric.data import HeteroData
#data = HeteroData(torch.load('factorization_machines_dataset.pt'))
train_data = HeteroData(torch.load(ROOT_FOLDER+'FactorizationMachines_Dataset_train_labelencoded_v2.pt'))
val_data = HeteroData(torch.load(ROOT_FOLDER+'FactorizationMachines_Dataset_val_labelencoded_v2.pt'))

/home/amos/mambaforge/envs/pyg_torch21/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
train_data['people'].labelencoding.shape

torch.Size([56])

In [5]:
train_data

HeteroData(
  _global_store={},
  people={
    x=[293444, 56],
    labelencoding=[56],
  },
  courses_and_programs={
    x=[55796, 26],
    labelencoding=[26],
  },
  (people, completed, courses_and_programs)={
    edge_index=[2, 360300],
    edge_label_index=[2, 154413],
  }
)

In [11]:
import torch
import torch.nn.functional as F
from models.TransE import TransE
from models.DistMult import DistMult
from models.FactorizationMachineModel import FactorizationMachineModel
import torch_geometric
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class Model(torch.nn.Module):
    def __init__(self, fm : torch.nn.Module, head, node_types, edge_types, ggn_output_dim, pnorm=1, num_supervisors=0, num_organizations=0):
        super().__init__()
        # edge_type onehot lookup table with keys
        # node_type onehot lookup table with keys
        self.node_type_embedding = torch.nn.Embedding(len(node_types), ggn_output_dim) # hidden channels should be the output dim of gnn
        self.num_supervisors = num_supervisors
        self.num_organizations = num_organizations
        self.edge_types = edge_types
        for edge_type in edge_types:
            if edge_type[1].startswith('rev_'):
                self.edge_types.remove(edge_type)
        
        # create edge to int mapping
        self.edgeindex_lookup = {edge_type:torch.tensor(i)  for i, edge_type in enumerate(edge_types)}
            
        if head=='TransE': 
            self.head = TransE(len(node_types), len(edge_types) , ggn_output_dim, p_norm= pnorm)  # KGE head with loss function
        elif head=='DistMult':
            self.head = DistMult(len(node_types), len(edge_types) , ggn_output_dim, p_norm= pnorm)  # KGE head with loss function
        else:
            raise NotImplementedError
        
        self.fm = fm
        
    

    def forward(self, hetero_data, edge_label_index, edge_label):
        
    
        people = hetero_data['people'].x[edge_label_index[0,:]]
        # last two columns in people are the indices of onehot, so change them to full onehot supervisor and organization
        #supervisors = torch.nn.functional.one_hot(people[:,-2].to(torch.int64), num_classes=self.num_supervisors).to(torch.float32)
        #organizations = torch.nn.functional.one_hot(people[:,-1].to(torch.int64), num_classes=self.num_organizations).to(torch.float32)
        #people = torch.cat((people[:,:-2], supervisors, organizations), dim=1)
        
                        
        learnings = hetero_data['courses_and_programs'].x[edge_label_index[1,:]]
    
        x = torch.cat((people,learnings),dim=1)
        scores = self.fm(x)
        pos_scores = scores[edge_label==1]
        neg_scores = scores[edge_label==0]
            

        return F.margin_ranking_loss(
            pos_scores,
            neg_scores,
            target=torch.ones_like(pos_scores), # 1 for similarity, -1 for dissimilarity
            margin=0.2
        )
        
    
out_channels = 1
hidden_channels = 16
num_heads = 0
num_layers = 0
pnorm = 2
head = 'TransE'
#gnn = HGT(hidden_channels=out_channels, out_channels=out_channels, num_heads=num_heads, num_layers=num_layers, node_types=train_data.node_types, data_metadata=metadata)
filename = 'HeteroData_Learnings_normalized_triangles_withadditionaldata_v1.pt'
data_forlookup = HeteroData.from_dict(torch.load(ROOT_FOLDER+filename))
num_supervisors = data_forlookup['people'].num_nodes
num_organizations = data_forlookup['organizations'].num_nodes
metadata = data_forlookup.metadata()
# add selfloops
for node_type in data_forlookup.node_types:
    metadata[1].append((node_type, 'self_loop', node_type))  
    
    

del data_forlookup
print(train_data['people'].labelencoding.shape, train_data['courses_and_programs'].labelencoding.shape)  
field_dims = torch.cat((train_data['people'].labelencoding,train_data['courses_and_programs'].labelencoding), dim=0)
print(field_dims)
# convert the field dims to integer
field_dims = field_dims.to(torch.int64)
fm = FactorizationMachineModel(
    field_dims=field_dims,
        embed_dim=hidden_channels)







model = Model(fm, head=head, node_types=metadata[0], edge_types=metadata[1], ggn_output_dim=out_channels, pnorm=pnorm, num_supervisors=num_supervisors, num_organizations=num_organizations)
#torch_geometric.compile(model, dynamic=True)
model.to(device)



torch.Size([56]) torch.Size([26])
tensor([293444,  55638, 264052, 264052, 264052, 264052, 264052, 264052, 264052,
        264052, 264052, 264052, 264052, 264052, 264052, 264052, 264052, 264052,
        264052, 264052, 264052, 264052, 264052, 264052, 264052, 264052, 264052,
        264052, 264052, 264052, 264052, 264052, 264052, 264052, 264052, 264052,
        264052, 264052, 264052, 264052, 264052, 264052, 264052, 264052, 264052,
        264052, 264052, 264052, 264052, 264052, 264052, 264052, 293444, 293444,
         13613,  13613,  55796, 264052, 264052, 264052, 264052, 264052, 264052,
        264052, 264052, 264052, 264052, 264052, 264052, 264052, 264052, 264052,
        264052, 264052, 264052, 264052, 264052, 264052, 264052, 264052, 264052,
        264052], device='cuda:0')


TypeError: can't convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.

In [7]:
56

56

In [8]:
field_dims

tensor([293444,  55638, 264052, 264052, 264052, 264052, 264052, 264052, 264052,
        264052, 264052, 264052, 264052, 264052, 264052, 264052, 264052, 264052,
        264052, 264052, 264052, 264052, 264052, 264052, 264052, 264052, 264052,
        264052, 264052, 264052, 264052, 264052, 264052, 264052, 264052, 264052,
        264052, 264052, 264052, 264052, 264052, 264052, 264052, 264052, 264052,
        264052, 264052, 264052, 264052, 264052, 264052, 264052, 293444, 293444,
         13613,  13613,  55796, 264052, 264052, 264052, 264052, 264052, 264052,
        264052, 264052, 264052, 264052, 264052, 264052, 264052, 264052, 264052,
        264052, 264052, 264052, 264052, 264052, 264052, 264052, 264052, 264052,
        264052])

In [9]:
fm

FactorizationMachineModel(
  (embedding): FeaturesEmbedding(
    (embedding): Embedding(20822892, 16)
  )
  (linear): FeaturesLinear(
    (fc): Embedding(20822892, 1)
  )
  (fm): FactorizationMachine()
)

In [10]:
from tqdm.auto import tqdm
from datetime import datetime
batch_size = 32

learning_rate = 2e-4
# torch get optimizer by string name
optimizer = 'Adam'
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) #2e-15


# create a tensorboard writer
from torch.utils.tensorboard import SummaryWriter
neighbors = '_'.join([str(n) for n in []])


timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

writer = SummaryWriter(ROOT_FOLDER+f'runs/learningpeople_factorizationmachines_{timestamp}_pnorm{pnorm}_lr{learning_rate}_bs{batch_size}_neighbors_{neighbors}_head_{head}_hiddenchannels_{hidden_channels}_outchannels_{out_channels}_numheads_{num_heads}_numlayers_{num_layers}')
print('writer',ROOT_FOLDER+f'runs/learningpeople_factorizationmachines_{timestamp}_pnorm{pnorm}_llr{learning_rate}_bs{batch_size}_neighbors_{neighbors}_head_{head}_hiddenchannels_{hidden_channels}_outchannels_{out_channels}_numheads_{num_heads}_numlayers_{num_layers}')





num_learnings = train_data['courses_and_programs'].num_nodes
neg_sample_ratio = 1
train_loader = nf_loader(train_data['people','completed','courses_and_programs'].edge_label_index.to('cpu'), batch_size, num_learnings, neg_sample_ratio)
val_loader = nf_loader(val_data['people','completed','courses_and_programs'].edge_label_index.to('cpu'), batch_size, num_learnings, neg_sample_ratio)
total_minibatches = get_total_minibatch_count_fm(batch_size, train_data['people','completed','courses_and_programs'].edge_label_index)


model.train()
start_epoch = 1
for epoch in range(start_epoch, start_epoch+1000):
    for i, (batch_edge_label_index, labels) in tqdm(enumerate(train_loader), total=total_minibatches):
        
        optimizer.zero_grad() 
        # batching is different depending on if node types in edge are same or different
        
        loss = model(train_data.to(device), batch_edge_label_index.to(device), labels.to(device))
        
        loss.backward()
        optimizer.step()
        
        total_samples_seen = i * batch_size
        writer.add_scalar('Loss/train', loss.item(), total_samples_seen)
        
        if i == total_minibatches-1:
            print(f'{i} loss: {loss.item():.4f}')
            writer.add_scalar('Epoch Loss/train', loss.item(), total_samples_seen)
        
        # print loss and minibatch in the same line
        print(f'{i} loss: {loss.item():.4f}', end='\r')
        
        if i % 300 == 0 or i == total_minibatches-1:
            model.eval()
            with torch.no_grad():
                val_loss = 0
                for _ in range(3):
                    try:
                        batch_edge_label_index, labels = next(val_loader)
                    except StopIteration:
                        val_loader = iter(val_loader)
                        batch_edge_label_index, labels = next(val_loader)
                    val_loss = model(val_data.to(device), batch_edge_label_index.to(device), labels.to(device))
                    
                    
            val_loss /= 3
            if i == 0:
                writer.add_scalar('Epoch Loss/val', val_loss, total_samples_seen)
                writer.add_scalar('Loss/val', val_loss, total_samples_seen)
            elif i == total_minibatches-1:
                writer.add_scalar('Epoch Loss/val', val_loss, total_samples_seen)
            else:
                writer.add_scalar('Loss/val', val_loss, total_samples_seen)
            

            print(f'val_loss: {val_loss:.4f}', end='\r')
            model.train()

        writer.flush()
        
        if i % 1000 == 0 or i == total_minibatches-1:
            folder = 'models'
            if not os.path.exists(folder):
                os.makedirs(folder)
            
            run_folder = ROOT_FOLDER+f'{folder}/learningpeople_factorizationmachines_{timestamp}_pnorm{pnorm}_llr{learning_rate}_bs{batch_size}_neighbors_{neighbors}_head_{head}_hiddenchannels_{hidden_channels}_outchannels_{out_channels}_numheads_{num_heads}_numlayers_{num_layers}'
            if not os.path.exists(run_folder):
                os.makedirs(run_folder)
                
            print('saving model to', run_folder)
            # save model and optimizer
            is_epoch = f'Ep{epoch}_' if i == total_minibatches-1 else ''
            torch.save({
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                }, run_folder+f'/{is_epoch}model_samplesseen{total_samples_seen}.pt')
            
writer.close()

writer runs/learningpeople_factorizationmachines_20231103_195322_pnorm2_llr0.0002_bs32_neighbors__head_TransE_hiddenchannels_16_outchannels_1_numheads_0_numlayers_0


  0%|          | 0/4826 [00:00<?, ?it/s]

torch.Size([64, 82]) tensor([[149242,      0,      0,  ...,      0,      0,      0],
        [244672,   3844,   7448,  ...,      0,      0,      0],
        [283744,      0,      0,  ...,      0,      0,      0],
        ...,
        [201083,   3692,   7554,  ...,      0,      0,      0],
        [161913,      0,      0,  ...,      0,      0,      0],
        [137156,  10039,    721,  ...,      0,      0,      0]],
       device='cuda:0')
torch.Size([64, 82]) tensor([[ 50480,   4428,      0,  ...,      0,      0,      0],
        [247290,  14171,    721,  ...,      0,      0,      0],
        [214730,  23181,      0,  ...,      0,      0,      0],
        ...,
        [ 17080,  27869,    721,  ...,      0,      0,      0],
        [165403,  23106,    721,  ...,      0,      0,      0],
        [161241,   3874,    200,  ...,      0,      0,      0]],
       device='cuda:0')
torch.Size([64, 82]) tensor([[255339,  23110,    721,  ...,      0,      0,      0],
        [ 15120,  27869,    7

  0%|          | 1/4826 [00:16<22:31:53, 16.81s/it]

torch.Size([64, 82]) tensor([[163972,  23362,    721,  ...,      0,      0,      0],
        [236234,  23353,   4277,  ...,      0,      0,      0],
        [253213,      0,      0,  ...,      0,      0,      0],
        ...,
        [262788,      0,      0,  ...,      0,      0,      0],
        [241937,    361,   3339,  ...,      0,      0,      0],
        [230249,   3666,    200,  ...,      0,      0,      0]],
       device='cuda:0')


...

torch.Size([64, 82]) tensor([[ 58467,      0,      0,  ...,      0,      0,      0],
        [286496,   9502,   1513,  ...,      0,      0,      0],
        [203943,  27702,      0,  ...,      0,      0,      0],
        ...,
        [256237,    555,   2561,  ...,      0,      0,      0],
        [231737,      0,      0,  ...,      0,      0,      0],
        [249314,      0,      0,  ...,      0,      0,      0]],
       device='cuda:0')


  0%|          | 6/4826 [00:27<3:41:46,  2.76s/it]

torch.Size([64, 82]) tensor([[261766,   3692,   7554,  ...,      0,      0,      0],
        [173181,      0,      0,  ...,      0,      0,      0],
        [199930,   3025,   7554,  ...,      0,      0,      0],
        ...,
        [203330,   8734,  12702,  ...,      0,      0,      0],
        [163245,    282,    721,  ...,      0,      0,      0],
        [248462,      0,      0,  ...,      0,      0,      0]],
       device='cuda:0')


  0%|          | 6/4826 [00:29<6:31:10,  4.87s/it]


KeyboardInterrupt: 